
<center>
<h4>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación</h4>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones</h3>
 <h2>Mentoría: Clasificación de Tumoresferas </h2>
</center>



<a name="exploratory_data_analysis"></a>
# **Práctico de Aprendizaje Supervisado**



Importamos las librerías necesarias:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

sns.set_context('talk')

import warnings
warnings.filterwarnings("ignore")


En este práctico, utilizaremos el archivo original *fiji_datos_0al7mo_labels.csv* que se encuentra en la carpeta *data/raw/*.

Además sumaremos los siguientes datos sintéticos que se encuentran en la carpeta *data/datos_sinteticos/*:
  * datos_sinteticos_dias_3_y_5.csv
  * synthetic_3y5_sint2.csv
  * synthetic_data_dia_3_si.csv
  * synthetic_data_dia_4_si.csv
  * synthetic_data_dia_5_si.csv

Como también dos archivos extras que están en la capeta *data/03_AS/*:
  * fiji_datos_mean_diam.csv
  * fiji_datos_noise.csv
  

## Regresión

$1.$   Utilizando del día 1 al 5 los datos clasificados como 'Esferoide' = 'si', realizar un ajuste del diámetro medio. Sean los datos reales *mean_diam_df* y la señal ruidosa *df_noise*.


In [2]:
mean_diam_df = pd.read_csv("data/03_AS/fiji_datos_mean_diam.csv")
mean_diam_df

FileNotFoundError: [Errno 2] No such file or directory: 'data/03_AS/fiji_datos_mean_diam.csv'

In [ ]:
df_noise = pd.read_csv("data/03_AS/fiji_datos_noise.csv")
#df_noise

In [ ]:
X_noise = df_noise['dia']
y_noise = df_noise['mean']
X_noise = np.array(X_noise.to_list())
y_noise = np.array(y_noise.to_list())

X_mean = mean_diam_df['dia']
y_mean = mean_diam_df['mean']
X_mean = np.array(X_mean.to_list())
y_mean = np.array(y_mean.to_list())


plt.scatter(X_noise, y_noise, color="blue", label="noise")
plt.scatter(X_mean, y_mean, color="yellow", label="mean")
plt.show()

Probar un ajuste lineal, polinomial, probar el ajuste "óptimo" que da el menor error cuadrático y sobreajuste.  

## Análisis para mean_diam_df

### Regresión lineal

In [ ]:
# Separamos las características y la variable objetivo
X = mean_diam_df[['dia']]  
y = mean_diam_df['mean']

# Creamos un modelo de regresión lineal
model = LinearRegression()

# Entrenamos el modelo con los datos
model.fit(X, y)

# Realizamos predicciones con el modelo
y_pred = model.predict(X)

# Visualizamos los resultados
plt.scatter(X, y, color='gold', label='Datos reales')
plt.plot(X, y_pred, color='darkorange', linewidth=2, label='Regresión lineal')
plt.xlabel('Día')
plt.ylabel('Diámetro medio')
plt.legend(fontsize='small', loc='upper left')
plt.show()

coeficiente_pendiente = round(model.coef_[0], 1)
termino_independiente = round(model.intercept_, 1)

print("Pendiente:", coeficiente_pendiente)
print("Término independiente:", termino_independiente)


In [ ]:
# Calculamos las predicciones del modelo
y_pred = model.predict(X_test)

# Calculamos el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)

mse_rounded = round(mse, 1)

print("Error cuadrático medio:", mse_rounded)

### Regresión polinomial

In [ ]:
train_errors = []

degrees = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

for degree in degrees:
    # Entreno al modelo para cada uno de los grados en degrees:
    pf = PolynomialFeatures(degree)
    lr = LinearRegression(fit_intercept=False)
    model = make_pipeline(pf, lr)
    model.fit(X, y)

    # predigo los valores con el modelo:
    y_pred = model.predict(X)

    # evaluo el error:
    train_error = round(mean_squared_error(y, y_pred),2)

    # Armo el array con los errores
    train_errors.append(train_error)

Calculamos el MSE en función del grado del polinomio

In [ ]:
for degree, train_error in zip(degrees, train_errors):
    print(degree, train_error)

Graficamos en función del grado del polinomio.

In [ ]:
plt.plot(degrees, train_errors, color="gold")
plt.xlim(0,11)
plt.ylim(0,1200)
plt.xlabel("Grado de Polinomio", size= 15)
plt.ylabel("MSE", size= 15)
plt.title("Error Cuadrático Medio (MSE) vs. Grado de Polinomio", pad=20, size= 15)
plt.show()

El mse disminuye abruptamente a partir del grado 1 al 5, dando indicios de sobreajuste a partir del grado 5, donde el MSE es 0.

In [ ]:
# Encontramos el índice del grado de polinomio con el menor error en el conjunto de evaluación
best_degree_index = np.argmin(train_errors)
best_degree = degrees[best_degree_index]

In [ ]:
# Imprimimos los errores cuadráticos medios para el mejor grado de polinomio
print("Mejor grado de polinomio:", best_degree)
print("MSE:", train_errors[best_degree_index])

Tal como podemos ver en la tabla de MSE (errores cuadráticos medios) como gráficamente, el grado del polinomio que reduce los errores y mejor ajusta los datos, es el polinomio de grado 5. El ajuste mejora considerablemente respecto a la regresión lineal dado que se encontraron MSE de menor valor que para la regresión lineal.

No obstante, podríamos pensar que la regresión lineal es un mejor modelo para aplicar a nuestros datos. Cuando el MSE es nulo, se origina un sobreajuste de nuestros datos. Esto ocurre a partir del grado 5 en la regresión polinomial. Ahora bien, el MSE para el polinomio de grado 4 tiene un valor de 26,9 (sería el MSE de menor valor sin sobreajuste), mientras que el MSE de la regresión lineal es 15,9. Teniendo esto en consideración, se podría seleccionar como mejor modelo el de regresión lineal para el ajuste de nuestros datos.

## Análisis para df_noise

### Regresión lineal

In [ ]:
# Separamos las características y la variable objetivo
X = df_noise[['dia']]  
y = df_noise['mean']

# Creamos un modelo de regresión lineal
model = LinearRegression()

# Entrenamos el modelo con los datos
model.fit(X, y)

# Realizamos predicciones con el modelo
y_pred = model.predict(X)

# Visualizamos los resultados
plt.scatter(X, y, color='gold', label='Datos reales')
plt.plot(X, y_pred, color='darkorange', linewidth=2, label='Regresión lineal')
plt.xlabel('Día')
plt.ylabel('Diámetro medio')
plt.legend(fontsize='small', loc='upper left')
plt.show()

coeficiente_pendiente = round(model.coef_[0], 1)
termino_independiente = round(model.intercept_, 1)

print("Pendiente:", coeficiente_pendiente)
print("Término independiente:", termino_independiente)

In [ ]:
# Calculamos las predicciones del modelo
y_pred = model.predict(X_test)

# Calculamos el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)

mse_rounded = round(mse, 1)

print("Error cuadrático medio:", mse_rounded)

Los datos ruidosos originan que el ajuste del modelo empeore, por lo que el MSE aumenta considerablemente para la regresión lineal con respecto a los datos sin ruido.

### Regresión polinomial

In [ ]:
train_errors = []

degrees = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

for degree in degrees:
    # Entreno al modelo para cada uno de los grados en degrees:
    pf = PolynomialFeatures(degree)
    lr = LinearRegression(fit_intercept=False)
    model = make_pipeline(pf, lr)
    model.fit(X, y)

    # predigo los valores con el modelo:
    y_pred = model.predict(X)

    # evaluo el error:
    train_error = round(mean_squared_error(y, y_pred),2)

    # Armo el array con los errores
    train_errors.append(train_error)

Calculamos los MSE para cada grado del polinomio.

In [ ]:
for degree, train_error in zip(degrees, train_errors):
    print(degree, train_error)

In [ ]:
plt.plot(degrees, train_errors, color="gold")
plt.xlim(0,11)
plt.ylim(0,1200)
plt.xlabel("Grado de Polinomio", size= 15)
plt.ylabel("MSE", size= 15)
plt.title("Error Cuadrático Medio (MSE) vs. Grado de Polinomio", pad=20, size= 15)
plt.show()

En este caso podemos observar que los MSE para los grados del 0 al 5 son un poco menores que para los datos sin ruido. No obstante, no se observa que el MSE se haga cero para los polinomios de mayor grado como sí sucede para aquellos datos sin ruido. 

### División en Entrenamiento y Evaluación
Dividimos aleatoriamente los datos en 80% para entrenamiento y 20% para evaluación:

In [ ]:
# Analizamos primero el caso de los datos no ruidosos.
X = mean_diam_df['dia'].values.reshape(-1, 1)
y = mean_diam_df['mean'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de X_test:", X_test.shape)
y_train


## Clasificación

Aquí vamos a usar el dataset sin modificar, que se encuentra en 'raw/fiji_datos_0al7mo_labels.csv', en la carpeta del [repositorio](https://github.com/luciabarg/datos_tumoresferas/tree/main/data/datos_sinteticos) y de la [carpeta compartida](https://drive.google.com/drive/folders/1RqGNySwACN33Qopmw0nHmj5Yv4M78ZXi?usp=drive_link).

1) Teniendo en cuenta la naturaleza de nuestros datos, es adecuado plantearlo como un problema supervisado? De qué tipo? Justificar.
   
2) El dataset que tenemos, es linealmente separable?

3) Explorar las características de los datos sintéticos generados, comparar con los datos que tenemos de los días 3,4 y 5. Los datos se encuentran en la carpeta del [repositorio](https://github.com/luciabarg/datos_tumoresferas/tree/main/data/datos_sinteticos) y de la [carpeta compartida](https://drive.google.com/drive/folders/1RqGNySwACN33Qopmw0nHmj5Yv4M78ZXi?usp=drive_link):

4) Elegir uno de las siguientes situaciones y generar su correspondiente dataset (leer hasta el final del enunciado antes de generarlos):

  * Tomar todo el dataset.

  * Realizar clasificación con solo los datos del día 3, 4 y 5.

  * Tomar solo los días 3,4 y 5 y sumar los datos sintéticos.
  
  * Utilizando sólo PCA (si quieren de todos días o solo de los días 3,4 y 5, con o sin datos sintéticos) pero indicar cuál se tomó.

\
**Elegir un escenario**, recordar eliminar una de las columnas altamente correlacionadas y también aquellas que no suman al problema. Si hubiera una columna constante, también deberían excluirla. Separen en conjuntos de entrenamiento y test, hacer tratamiento de ouliers, agregar columnas preprocesadas al original como algunas que sean combinaciones/modificaciones de las demás, multiplicaciones, logaritmos, potencias, [por ej](https://docs.google.com/presentation/d/e/2PACX-1vSLfKgsq-NuF2aWQF3OPkgLvBn25A2khGh0QIJkpFb6QgRZ7dGne32GEkTiC4M6yg/pub?start=false&loop=false&delayms=3000&slide=id.gb81ac3e375_0_32), verificando que no sean correlacionadas con las que ya tienen.


**Recordar que las transformaciones se realizan sobre el conjunto de train y luego se ajusta el de test**. Si hacen todo antes y luego separan, puede haber traspaso de información sobre el conjunto de train al test (*data leakage*).


Realizar un EDA rápido de como quedaron las variables y si la distribuciones en test son similares a las que tienen en train.
En todos los casos, tanto para train y test, visualizar la cantidad de datos para cada clase y calcular el porcentaje de las mismas.

Tenemos pocos datos, es muy posible que se genere overfitting. Cómo podrían tratar de solventar este inconveniente? Implementar si es posible.

Implementar sobre los conjuntos *test* y *train* del escenario elegido algún clasificador lineal como también probar DT, Random Forest y XGBoost.

Explorar con parámetros de defecto (modelo baseline) y con búsqueda de hiperparámetros y por medio de las diferentes métricas (sobre todo f1 y precision) determinar cuál es el mejor algoritmo de clasificación.
En los algoritmos que lo permitan, hacer listado de importancia de features y probar con diferentes combinaciones de columnas si la métrica hallada mejora si se disminuye la cantidad de columnas (puede ser que esto no ocurra también).

Con el algoritmo que tenga las mejores métricas , probar si mejora o empeoran la clasificación probando algún otro escenario (con todos los pasos que implica).

\
OPCIONAL:


Probar la métrica: coeficiente de correlación de [Matthews](https://bmcgenomics.biomedcentral.com/articles/10.1186/s12864-019-6413-7) (está implementado en sklearn como una métrica más: *from sklearn.metrics import matthews_corrcoef* )

Si se animan, tenemos 5 días. Probar clasificación multiclase para identificar a q día se corresponden las muestras. O tomar solo los 3 que usaron en los escenarios anteriores.

Super buenas prácticas en este [repositorio](https://github.com/daianadte/wids-cba-2023/), es muy instructivo chusmear sobre todo el archivo  ['06_FinalModel.ipynb'](https://github.com/daianadte/wids-cba-2023/blob/main/06_FinalModel.ipynb) solo para que vean un análisis posterior a implementar los modelos y que existen los Shap Values, que es una técnica utilizada para explicar las predicciones.


----
Como este es el último práctico, si se animan, podrían implementar clústering.
Pueden implementar kmeans sobre el dataset normalizado, usando PCA, o incluso animarse a probar t-SNE y UMAP.

Realizar EDA sobre los conjuntos que encontraron y traten de explicarlos.

Muchas veces en problemas de negocios, no es tan importante el modelo en sí, qué tan bien separa los datos sino la interpretabilidad que se puede dar a los resultados. Podríamos coordinar con Luciano para charlar si lo que encontraron a partir de las agrupaciones, es coherente o no, sería como parte de charlar con el "cliente" a ver si está de acuerdo a lo hallado. O sea, pueden encontrar explicaciones a los datos distintos que la de "esferoides".


### $1.$ Teniendo en cuenta la naturaleza de nuestros datos, es adecuado plantearlo como un problema supervisado? De qué tipo? Justificar.

Nuestros datos se adecuan a un problema de aprendizaje supervisado debido a que contamos con etiquetas previamente asignados a cada instancia de datos. En este caso, nuestra variable objetivo 'esferoide' es binaria, tomando solo dos valores posibles: "sí" o "no". Esto indica que estamos abordando un problema de clasificación binaria.

El aprendizaje supervisado implica que nuestro modelo aprenderá a partir de ejemplos de entrada y sus correspondientes etiquetas de salida, con el objetivo de realizar predicciones precisas sobre nuevas instancias de datos no etiquetados. En este contexto, estamos entrenando un modelo para predecir si una instancia pertenece a una de las dos clases: "sí" o "no", lo que confirma la naturaleza de clasificación binaria de nuestro problema.

En resumen, nuestro problema se ajusta a un problema de aprendizaje supervisado de clasificación binaria, ya que contamos con etiquetas y el objetivo es predecir la clase de nuevas instancias en una de las dos categorías posibles: "sí" o "no".

### $2.$ El dataset que tenemos, es linealmente separable?.

In [ ]:
# Levantamos el archivo
url = 'https://raw.githubusercontent.com/luciabarg/datos_tumoresferas/main/data/raw/fiji_datos_0al7mo_labels.csv'
df_fiji_datos = pd.read_csv(url)
df_fiji_datos.head()

Repitamos el análisis de PCA realizado para el tp anterior.

In [ ]:
np.random.seed(0)

# Standarization
cols_to_project = ['Area', 
                         'Perim.',
                         #'Width',
                         #'Height',
                         'Circ.',
                         'Feret',
                         'MinFeret',
                         'AR',
                         'Round',
                         'Solidity',
                         #'Esferoide',
                         'Diameter',
                         'n_diam']

df_standard = df_fiji_datos[cols_to_project].copy()

# Standardize the data
df_standard = (df_standard - df_standard.mean()) / df_standard.std()
pca = PCA(n_components=10)
pca.fit(df_standard)

df_projected = pca.transform(df_standard)

plt.figure(figsize=(10, 6))

g = sns.JointGrid(x=df_projected[:, 0], y=df_projected[:, 1],hue=df_fiji_datos['Esferoide'], height=6)
g.plot_joint(sns.scatterplot, s=100)
g.plot_marginals(sns.stripplot)
plt.tight_layout()  # Añadir esta línea para ajustar el gráfico
plt.show()    



Del gráfico de las dos componentes principales de PCA podríamos decir que a simple vista los datos parecerían ser linealmente separables.

### $3.$ Explorar las características de los datos sintéticos generados, comparar con los datos que tenemos de los días 3,4 y 5. Los datos se encuentran en la carpeta del repositorio y de la carpeta compartida.

In [ ]:
#...

$4.$ Elegir uno de las siguientes situaciones y generar su correspondiente dataset (leer hasta el final del enunciado antes de generarlos):

* Tomar todo el dataset.

* Realizar clasificación con solo los datos del día 3, 4 y 5.

* Tomar solo los días 3,4 y 5 y sumar los datos sintéticos.
  
* Utilizando sólo PCA (si quieren de todos días o solo de los días 3,4 y 5, con o sin datos sintéticos) pero indicar cuál se tomó.

In [ ]:
#...